In [10]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import ast
import jellyfish
from util import *

In [3]:
df_games_sales = load_dataframes("vgsales_cleand")
df_games_vote = load_dataframes("games_of_all_time_cleand")

In [178]:
print(len(df_games_sales))
df_games_sales.head(10)

16598


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
5,6,Tetris,GB,1989,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26
6,7,New Super Mario Bros.,DS,2006,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01
7,8,Wii Play,Wii,2006,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02
8,9,New Super Mario Bros. Wii,Wii,2009,Platform,Nintendo,14.59,7.06,4.70,2.26,28.62
9,10,Duck Hunt,NES,1984,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31


In [7]:
print(len(df_games_vote))
df_games_vote.head()

8831


,Name,Meta_score,User_score,Platform,Description,URL,Developer,Genre,Type,Rating
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,['nintendo-64'],"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,"['Action Adventure', 'Fantasy']",singleplayer,E
1,Super Mario Galaxy,97.0,91.0,['wii'],[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E
2,Super Mario Galaxy 2,97.0,91.0,['wii'],"Super Mario Galaxy 2, the sequel to the galaxy...",https://www.metacritic.com/game/wii/super-mari...,Nintendo EAD Tokyo,"['Action', 'Platformer', '3D']",singleplayer,E
3,Metroid Prime,97.0,89.0,['gamecube'],Samus returns in a new mission to unravel the ...,https://www.metacritic.com/game/gamecube/metro...,Retro Studios,"['Action', 'Shooter', 'First-Person', 'Sci-Fi']",singleplayer,T
4,Super Mario Odyssey,97.0,89.0,['switch'],New Evolution of Mario Sandbox-Style Gameplay....,https://www.metacritic.com/game/switch/super-m...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E10+


In [33]:
df_games_sales["Platform"].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       '3DS', 'PS4', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne', 'GC',
       'WiiU', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [57]:
#df_games_vote["Platform"] = df_games_vote["Platform"].apply(ast.literal_eval)
platform_set = set()
for index_gv, row_gv in df_games_vote.iterrows():
    l = ast.literal_eval(str(row_gv["Platform"]))
    for p in l:
        platform_set.add(p)
   
print(platform_set)

0
{'gamecube', 'playstation-vita', 'xbox-one', 'stadia', 'playstation-5', 'psp', 'playstation-4', 'xbox-360', 'wii-u', 'nintendo-64', 'pc', 'xbox', '3ds', 'wii', 'playstation-3', 'game-boy-advance', 'playstation-2', 'ds', 'dreamcast', 'switch', 'playstation', 'xbox-series-x'}


In [58]:
platform_dict = {
    'nintendo-64':'N64', 
    'gamecube':'GC', 
    'pc':'PC', 
    '3ds':'3DS', 
    'playstation-vita':'PSV', 
    'wii':'Wii', 
    'playstation-3':'PS3', 
    'ds':'DS', 
    'dreamcast':'DC', 
    'xbox':'XB', 
    'playstation-2':'PS2', 
    'switch':"SWITCH", 
    'playstation':'PS', 
    'game-boy-advance':'GBA', 
    'playstation-4':'PS4', 
    'xbox-360':'X360', 
    'wii-u':'WiiU',
    'xbox-one':'XOne',
    'stadia':'PC',
    'playstation-5':'PS5',
    'psp':'PSP',
    'xbox-series-x':'XOneX'
}

In [180]:
df = df_games_vote.copy()
match_found = 0
names = df_games_sales["Name"].unique()
df["jaro_dst"] = 0.0
platform = []
for index_gv, row_gv in df.iterrows():
    best = ""
    best_score = 0
    for name in names:
        score = jellyfish.jaro_distance(row_gv["Name"], name)
        if(score == 1.0):
            best = name
            best_score = score
            break
        elif(score > 0.95 and score > best_score):
            best = name
            best_score = score
            
    df.loc[index_gv, "jaro_dst"] = best_score
    if(best_score > 0.95):
        match_found += 1
        df.loc[index_gv, "Name"] = best
    if(index_gv % 100 == 0):
        print(index_gv)
    
    #new_platform = []
    #l = ast.literal_eval(str(row_gv["Platform"]))
    #for p in l:
    #    new_platform += [platform_dict[p]]
    #platform.append(new_platform)

#df["Platform"] = platform
#df["Genre"] = df["Genre"].apply(lambda x:ast.literal_eval(str(x)))
print("Match found:" + str(match_found))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
Match found:3790


In [181]:
df

,Name,Meta_score,User_score,Platform,Description,URL,Developer,Genre,Type,Rating,jaro_dst
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,[nintendo-64],"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,"['Action Adventure', 'Fantasy']",singleplayer,E,1.0
1,Super Mario Galaxy,97.0,91.0,[wii],[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E,1.0
2,Super Mario Galaxy 2,97.0,91.0,[wii],"Super Mario Galaxy 2, the sequel to the galaxy...",https://www.metacritic.com/game/wii/super-mari...,Nintendo EAD Tokyo,"['Action', 'Platformer', '3D']",singleplayer,E,1.0
3,Metroid Prime,97.0,89.0,[gamecube],Samus returns in a new mission to unravel the ...,https://www.metacritic.com/game/gamecube/metro...,Retro Studios,"['Action', 'Shooter', 'First-Person', 'Sci-Fi']",singleplayer,T,1.0
4,Super Mario Odyssey,97.0,89.0,[switch],New Evolution of Mario Sandbox-Style Gameplay....,https://www.metacritic.com/game/switch/super-m...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E10+,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8826,kill.switch,74.0,80.5,"[xbox, playstation-2]","In a world on the brink of global conflict, yo...",https://www.metacritic.com/game/xbox/killswitch,Namco,"['Action', 'Shooter', 'Third-Person', 'Modern']",singleplayer,T,0.0
8827,nail'd,69.0,76.0,[pc],nail’d is all about eschewing boring realism f...,https://www.metacritic.com/game/pc/naild,Techland,"['Driving', 'Racing', 'Arcade', 'Automobile']",singleplayer,E10+,1.0
8828,oOo: Ascension,70.0,73.0,[xbox-one],oOo: Ascension is a lightning-paced test of re...,https://www.metacritic.com/game/xbox-one/ooo-a...,Kenny Creanor,"['Action', 'General', 'Arcade']",multiplayer,E,0.0
8829,uDraw Studio,71.0,71.0,[wii],"Packaged with the uDraw GameTablet, uDraw Stud...",https://www.metacritic.com/game/wii/udraw-studio,THQ,"['Miscellaneous', 'General']",NaN,E,1.0


In [188]:
merged = pd.merge(df.drop(columns=["Platform"]),df_games_sales,"right",on = "Name")
print(len(merged))
merged

16970


,Name,Meta_score,User_score,Description,URL,Developer,Genre_x,Type,Rating,jaro_dst,Rank,Platform,Year,Genre_y,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,"['Action Adventure', 'Fantasy']",singleplayer,E,1.000000,95,N64,1998,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
1,The Legend of Zelda: Ocarina of Time,94.0,90.0,The Legend of Zelda: Ocarina of Time 3D takes ...,https://www.metacritic.com/game/3ds/the-legend...,GREZZO,"['Miscellaneous', 'Fantasy', 'Compilation', 'A...",singleplayer,E10+,0.974359,95,N64,1998,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
2,The Legend of Zelda: Ocarina of Time,99.0,91.0,"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,"['Action Adventure', 'Fantasy']",singleplayer,E,1.000000,269,3DS,2011,Action,Nintendo,2.03,1.27,0.62,0.30,4.21
3,The Legend of Zelda: Ocarina of Time,94.0,90.0,The Legend of Zelda: Ocarina of Time 3D takes ...,https://www.metacritic.com/game/3ds/the-legend...,GREZZO,"['Miscellaneous', 'Fantasy', 'Compilation', 'A...",singleplayer,E10+,0.974359,269,3DS,2011,Action,Nintendo,2.03,1.27,0.62,0.30,4.21
4,Super Mario Galaxy,97.0,91.0,[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer,E,1.000000,49,Wii,2007,Platform,Nintendo,6.16,3.40,1.20,0.76,11.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16965,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16592,PSV,2016,Action,dramatic create,0.00,0.00,0.01,0.00,0.01
16966,Eiyuu Densetsu: Sora no Kiseki Material Collec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16593,PSP,2007,Role-Playing,Falcom Corporation,0.00,0.00,0.01,0.00,0.01
16967,Plushees,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16595,DS,2008,Simulation,Destineer,0.01,0.00,0.00,0.00,0.01
16968,Woody Woodpecker in Crazy Castle 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16596,GBA,2002,Platform,Kemco,0.01,0.00,0.00,0.00,0.01


In [189]:
merged[merged["Name"] == "Grand Theft Auto V"]

,Name,Meta_score,User_score,Description,URL,Developer,Genre_x,Type,Rating,jaro_dst,Rank,Platform,Year,Genre_y,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
4105,Grand Theft Auto V,96.8,81.4,Los Santos is a sprawling sun-soaked metropoli...,https://www.metacritic.com/game/xbox-one/grand...,Rockstar North,"['Action Adventure', 'Modern', 'Open-World']",multiplayer,M,1.0,17,PS3,2013,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40
4106,Grand Theft Auto V,96.8,81.4,Los Santos is a sprawling sun-soaked metropoli...,https://www.metacritic.com/game/xbox-one/grand...,Rockstar North,"['Action Adventure', 'Modern', 'Open-World']",multiplayer,M,1.0,24,X360,2013,Action,Take-Two Interactive,9.63,5.31,0.06,1.38,16.38
4107,Grand Theft Auto V,96.8,81.4,Los Santos is a sprawling sun-soaked metropoli...,https://www.metacritic.com/game/xbox-one/grand...,Rockstar North,"['Action Adventure', 'Modern', 'Open-World']",multiplayer,M,1.0,45,PS4,2014,Action,Take-Two Interactive,3.80,5.81,0.36,2.02,11.98
4108,Grand Theft Auto V,96.8,81.4,Los Santos is a sprawling sun-soaked metropoli...,https://www.metacritic.com/game/xbox-one/grand...,Rockstar North,"['Action Adventure', 'Modern', 'Open-World']",multiplayer,M,1.0,198,XOne,2014,Action,Take-Two Interactive,2.66,2.01,0.00,0.41,5.08
4109,Grand Theft Auto V,96.8,81.4,Los Santos is a sprawling sun-soaked metropoli...,https://www.metacritic.com/game/xbox-one/grand...,Rockstar North,"['Action Adventure', 'Modern', 'Open-World']",multiplayer,M,1.0,1898,PC,2015,Action,Take-Two Interactive,0.36,0.64,0.00,0.08,1.08


In [190]:
#platform = []
genre = []
for index, row in merged.iterrows():
    #if(not row["Platform_x"] is np.nan):
    #    l = row["Platform_x"].copy()
    #    l.append(row["Platform_y"])
    #    platform.append(list(set(l)))
    #else:
    #    platform.append([row["Platform_y"]])
        
    if(not row["Genre_x"] is np.nan):
        l = list(ast.literal_eval(str(row["Genre_x"])))
        l.append(row["Genre_y"])
        genre.append(list(set(l)))
    else:
        genre.append([row["Genre_y"]])
#merged["Platform"] = platform
merged["Genre"] = genre
merged.drop(columns=["Genre_x","Genre_y","jaro_dst"], inplace = True)
merged

,Name,Meta_score,User_score,Description,URL,Developer,Type,Rating,Rank,Platform,Year,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Genre
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,singleplayer,E,95,N64,1998,Nintendo,4.10,1.89,1.45,0.16,7.60,"[Action Adventure, Fantasy, Action]"
1,The Legend of Zelda: Ocarina of Time,94.0,90.0,The Legend of Zelda: Ocarina of Time 3D takes ...,https://www.metacritic.com/game/3ds/the-legend...,GREZZO,singleplayer,E10+,95,N64,1998,Nintendo,4.10,1.89,1.45,0.16,7.60,"[Compilation, Open-World, Miscellaneous, Actio..."
2,The Legend of Zelda: Ocarina of Time,99.0,91.0,"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,singleplayer,E,269,3DS,2011,Nintendo,2.03,1.27,0.62,0.30,4.21,"[Action Adventure, Fantasy, Action]"
3,The Legend of Zelda: Ocarina of Time,94.0,90.0,The Legend of Zelda: Ocarina of Time 3D takes ...,https://www.metacritic.com/game/3ds/the-legend...,GREZZO,singleplayer,E10+,269,3DS,2011,Nintendo,2.03,1.27,0.62,0.30,4.21,"[Compilation, Open-World, Miscellaneous, Actio..."
4,Super Mario Galaxy,97.0,91.0,[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,singleplayer,E,49,Wii,2007,Nintendo,6.16,3.40,1.20,0.76,11.52,"[Platform, Action, Platformer, 3D]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16965,Chou Ezaru wa Akai Hana: Koi wa Tsuki ni Shiru...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16592,PSV,2016,dramatic create,0.00,0.00,0.01,0.00,0.01,[Action]
16966,Eiyuu Densetsu: Sora no Kiseki Material Collec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16593,PSP,2007,Falcom Corporation,0.00,0.00,0.01,0.00,0.01,[Role-Playing]
16967,Plushees,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16595,DS,2008,Destineer,0.01,0.00,0.00,0.00,0.01,[Simulation]
16968,Woody Woodpecker in Crazy Castle 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16596,GBA,2002,Kemco,0.01,0.00,0.00,0.00,0.01,[Platform]


In [191]:
save_dataframes(merged,"merged_sales_ratings")